# KSCHOLL TFM


### Load the data 
First of all, load the needed libraries.

Geograpy usually give some issues during the installation, better use: 
`pip install geograpy3`

In [2]:
import datadotworld as dw
import pandas as pd
import re
import nltk
from dateparser.search import search_dates
import geograpy as gt
import numpy as np

#### Create conection with the Dataworld API and load the dataset with the URLS of all the datasets from WPP

In [3]:
client = dw.api_client()

#### Load the dataset with all URLs of WPP datasets

In [ ]:
dfD = pd.read_excel('https://query.data.world/s/gal5osq7zrw25gm5m265ogeyantsxt')

#### Load all the metadata from the datasets and save it in a dataframe 

In [5]:
datasets = client.fetch_contributing_datasets(limit = '100')
next_page = datasets.get('next_page_token')
ldatasets = datasets.get('records')

while next_page != None:
    datasets = client.fetch_contributing_datasets(limit = '100', next = next_page)
    next_page = datasets.get('next_page_token')
    ldatasets = ldatasets + datasets.get('records')
    
df = pd.DataFrame.from_dict(ldatasets[0], orient='index').transpose()
for i in range(1,len(ldatasets)):
    df = df.append(pd.DataFrame.from_dict(ldatasets[i], orient='index').transpose())
df = df.reset_index(drop = True)

### Prepare the data 

#### Start working with the metadata. First, extrat the tags and save it in a new df

In [8]:
dtagsl = []
for index, row in df.iterrows():
    for i in row.tags:
        dtagsl.append([row.id,i])
        
dftags = pd.DataFrame(dtagsl, columns = ['Name','Tag'])

#### Summary have a lot of info, we are going to extract and distribute it into new cols

In [10]:
for index, row in df.iterrows():
    s = str(row.summary)
    resultC = re.findall('\*\*(.*):\*\*', s)
    resultF = re.findall(':\*\*(.*)\**', s)
    n = 0
    for i in resultC:
        if i in df.columns:
            df.loc[index,i] = resultF[n]
            n += 1
        else:
            df[i] = None
            df.loc[index,i] = resultF[n]
            n += 1

#### Some cols are no representative because most of the are empty. We remove all the cols with less than 80% of data

In [11]:
nulls = df.isna().sum()
n = 0
colsToDrop = []
for i in nulls:
    if i/df.shape[0] > 0.80:
        colsToDrop.append(nulls.index[n])
        df = df.drop([nulls.index[n]] , axis = 1)
    n += 1

#### Find dates on the summary of each dataset.

#### If we find two dates we consider as starting and finish date of the data.

#### We save it in a new DF with name-year

In [12]:
for index, row in df.iterrows():
    if df.iloc[index].When != None:
        try:
            dates = search_dates(df.iloc[index].When.strip())
            firstDate = dates[0][1].year
            lastDate = dates[len(dates) - 1][1].year
            if firstDate < lastDate:
                df.loc[index,'MinD'] = firstDate
                df.loc[index,'MaxD'] = lastDate
            elif firstDate > lastDate:
                df.loc[index,'MinD'] = lastDate 
                df.loc[index,'MaxD'] = firstDate
            elif firstDate == lastDate:
                df.loc[index,'MinD'] = lastDate 
                df.loc[index,'MaxD'] = lastDate            
        except:
            None
            
ddates = []
for index, row in df.iterrows():
    if not np.isnan(row.MinD):
        if int(row.MinD) == int(row.MaxD):
            ddates.append([row.id, int(row.MinD)])            
        else:
            for i in range(int(row.MinD),int(row.MaxD) + 1):
                ddates.append([row.id, i])
                
dfdates = pd.DataFrame(ddates, columns = ['Name','Year'])

#### Load an online .csv with country name and compare with the ones that we haave extractec from the summaries.

#### Create a new DF with dataset name and dates included.

In [16]:
countries = pd.read_csv('https://raw.githubusercontent.com/datasets/country-list/master/data.csv')

dcountries = []
for index, row in df.iterrows():
    try:
        places = gt.get_place_context(text = df.iloc[index].summary)
        for i in places.country_mentions:
            if i[0] in list(countries.Name):
                dcountries.append([row.id, i[0]])
    except:
        None
        
dfcountries = pd.DataFrame(dcountries, columns = ['Name','Country'])

#### Create a new col with the number of files from each dataset. 

In [18]:
for index, rows in df.iterrows():
    df.loc[index, 'nfiles'] = len(df.files[index])

#### To have a good final DF we need to merge all the df that we have.

#### Merge by name doesn't works with all the dataset, we have to merge also by title and manage the duplicated cols

In [22]:
df = df.rename(columns = {'id' : 'Name'})
dfF = pd.merge(df,dfD, on = 'Name' , how = 'left')
dfF = pd.merge(dfF,dfD, left_on = 'title' , right_on = 'Name', how = 'left')

dfF["URL_x"] = dfF["URL_x"].fillna(dfF["URL_y"])
dfF["Collection_x"] = dfF["Collection_x"].fillna(dfF["Collection_y"])
dfF["Topic _x"] = dfF["Topic _x"].fillna(dfF["Topic _y"])
dfF["Source Type_x"] = dfF["Source Type_x"].fillna(dfF["Source Type_y"])
dfF["Privacy_x"] = dfF["Privacy_x"].fillna(dfF["Privacy_y"])

## Database transfer

#### We have to storage all the data to can work with the visualization, SQL is a good option.

#### Import the needed libraries. 

#### Install progressbar with `pip install progressbar2`

#### Install PyMySQL with `pip install PyMySQL``

In [ ]:
import time
import progressbar
import pymysql
from datetime import date

#### The first step y create the tables in the db for all the dataframes and one extra for a data summary

In [ ]:
def sql_tabletag(con):
    cursorObj.execute("CREATE TABLE dftags(title text , tag text)")
    con.commit()
    
def sql_tablefull(con):
    cursorObj = con.cursor()
    cursorObj.execute("CREATE TABLE full_data(owner text , Name text , title text, description text, summary text, tags text, visibility text, files text, status text,  created text, updated text, is_project text, access_level text,Data_Origin text, What text, When_ text, AutoSync text, Cite text, Who text, Source_Description text, MinD text, MaxD text, URL text, Collection text, Topic text, SourceType text, Privacy text)")
    con.commit()

def sql_tabledates(con):
    cursorObj.execute("CREATE TABLE dates(title text , date text)")
    con.commit()
    
def sql_tablecountries(con):
    cursorObj.execute("CREATE TABLE countries(title text , country text)")
    con.commit()

def sql_tablesummary(con):
    cursorObj = con.cursor()
    cursorObj.execute("CREATE TABLE summary(updated text , datasets INT , files INT, cat_sup INT, topics INT, tags INT, countries INT, MinD INT)")
    con.commit()
    

#### After have all the tables, we prepare function to upload data to the tables

In [ ]:
def insertDFtags(df, con):
    cursorObj = con.cursor()
    #Before add data, delete all the data in the table. Faster than review the existing data.
    cursorObj.execute('DELETE FROM dftags;',);
    con.commit()

    barposition=0
    with progressbar.ProgressBar(max_value=len(df)) as bar:
        cursorObj = con.cursor()
        for i,row in df.iterrows():
            arr = df.iloc[i]
            insertsql = ('INSERT INTO dftags VALUES'+("('"+arr[0]+"' , '"+arr[1]+"')"))
            cursorObj.execute(insertsql)
            con.commit()
            barposition+=1
            bar.update(i)
            
#Replace some special characters because it gives some issues when you try to insert into the SQL
def insertDFfull(df, con):
    cursorObj = con.cursor()
    try: 
        cursorObj.execute('DELETE FROM full_data;',);
        con.commit()
    except:
        pass
    barposition=0
    with progressbar.ProgressBar(max_value=len(df)) as bar:    
        cursorObj = con.cursor()
        for i,row in df.iterrows():
            arr = df.iloc[i]

            title = arr[2]
            try:
                title = title.replace("/","").replace("-","").replace("."," ").replace(","," ").replace(":","").replace("*","").replace("'","").replace("{","").replace("}","")
            except:
                title = str(arr[2])

            summary = arr[4]
            try:
                summary = summary.replace("/","").replace("-","").replace("."," ").replace(","," ").replace(":","").replace("*","").replace("'","").replace("{","").replace("}","") 
            except:
                summary = str(arr[4])

            descrip = arr[3]
            try: 
                descrip = descrip.replace("/","").replace("-","").replace("."," ").replace(","," ").replace(":","").replace("*","").replace("'","").replace("{","").replace("}","") 
            except:
                descrip = str(arr[3])

            source = arr[19]
            try: 
                source = source.replace("/","").replace("-","").replace("."," ").replace(","," ").replace(":","").replace("*","").replace("'","").replace("{","").replace("}","") 
            except:
                source = str(arr[19])

            cite = arr[17]
            try: 
                cite = cite.replace("/","").replace("-","").replace("."," ").replace(","," ").replace(":","").replace("*","").replace("'","").replace("{","").replace("}","") 
            except:
                cite = str(arr[17])


            whatt = arr[14]
            try: 
                whatt = whatt.replace("/","").replace("-","").replace("."," ").replace(","," ").replace(":","").replace("*","").replace("'","").replace("{","").replace("}","") 
            except:
                whatt = str(arr[14])

            #print(arr)
            insertsql = ('INSERT INTO full_data VALUES'+("('" +str(arr[0])+ "','" +str(arr[1])+ "','" +str(title)+ "','" +str(descrip)+"','" +summary+"','" +str((', '.join(arr[5])))+"','" +str(arr[6])+"','" +str(len(arr[7]))+"','" + str(arr[8]) +"','" + str(arr[9]) +"','" +str(arr[10])+"','" +str(arr[11])+"','" +str(arr[12])+"','" +str(arr[13])+"','" +str(whatt)+ "','" +str(arr[15])+"','" +str(arr[16])+"','" +cite+"','" +str(arr[18])+"','" +source+"','" +str(arr[20])+"','" +str(arr[21])+"','" +str(arr[23])+"','" +str(arr[24])+"','" +str(arr[25])+"','" +str(arr[26])+"','" +str(arr[27])+ "')" )) 
            #print (insertsql)
            cursorObj.execute(insertsql)
            con.commit()
            barposition+=1
            bar.update(i)

def summary(con):
    cursorObj = con.cursor()

    today = date.today()
    d1 = today.strftime("%d/%m/%Y")

    cursorObj = con.cursor()
    names_query = """SELECT Name from full_data"""
    cursorObj.execute(names_query)
    names_list = cursorObj.fetchall()

    names_query = """SELECT files from full_data"""
    cursorObj.execute(names_query)
    files_list = cursorObj.fetchall()
    totalfiles = 0
    files_list = list(files_list)
    for i in files_list:
        num = int(i[0])
        totalfiles = totalfiles+num

    names_query = """SELECT Collection from full_data"""
    cursorObj.execute(names_query)
    collection_list = cursorObj.fetchall()
    collection_list = list(set(collection_list))
    for i in range (len(collection_list)-1):
        if collection_list[i][0] == "nan":
            del collection_list[i]
    len(collection_list)

    topics_query = """SELECT Topic from full_data"""
    cursorObj.execute(topics_query)
    topic_list = cursorObj.fetchall()
    topic_list = list(set(topic_list))
    for i in range (len(topic_list)-1):
        if topic_list[i][0] == "nan":
            del topic_list[i]
    len(topic_list)

    tags_query = """SELECT tag from dftags"""
    cursorObj.execute(tags_query)
    tags_list = cursorObj.fetchall()
    tags_list = list(set(tags_list))
    for i in range (len(tags_list)-1):
        if tags_list[i][0] == "nan":
            del tags_list[i]
    len(tags_list)

    countries_query = """SELECT country from countries"""
    cursorObj.execute(countries_query)
    countries_list = cursorObj.fetchall()
    countries_list = list(set(countries_list))
    for i in range (len(countries_list)-1):
        if countries_list[i][0] == "nan":
            del countries_list[i]
    len(countries_list)

    minD_query = """SELECT date from dates"""
    cursorObj.execute(minD_query)
    minD_list = cursorObj.fetchall()
    minD_list = list(set(minD_list))
    mindnum = 2020
    for i in range (len(minD_list)):
        if minD_list[i][0] == "nan":
            pass
        else:
            num= int(float(minD_list[i][0]))
            #print (num)
            if num < mindnum:
                mindnum = num
    
    cursorObj.execute('DELETE FROM summary;',);
    con.commit()
    
    insertsql = ('INSERT INTO summary VALUES'+("('" +d1+ "','" +str(len(names_list))+ "','" +str(totalfiles)+ "','" +str(len(collection_list))+"','" +str(len(topic_list))+"','" +str(len(tags_list))+"','" +str(len(countries_list))+"','" +str(mindnum) + "')" )) 
    print (insertsql)
    cursorObj.execute(insertsql)
    print ('Resume updated to the SQL')
    con.commit()
    
def insertDFdates(df, con):
    cursorObj = con.cursor()
    cursorObj.execute('DELETE FROM dates;',);
    con.commit()
    barposition=0
    with progressbar.ProgressBar(max_value=len(df)) as bar:
        cursorObj = con.cursor()
        for i,row in df.iterrows():
            arr = df.iloc[i]
            insertsql = ('INSERT INTO dates VALUES'+("('"+str(arr[0])+"' , '"+str(arr[1])+"')"))
            cursorObj.execute(insertsql)
            con.commit()
            barposition+=1
            bar.update(i)

In [ ]:
def insertDFcountries(df, con):
    cursorObj = con.cursor()
    cursorObj.execute('DELETE FROM countries;',);
    con.commit()

    barposition=0
    with progressbar.ProgressBar(max_value=len(df)) as bar:
        cursorObj = con.cursor()
        for i,row in df.iterrows():
            arr = df.iloc[i]
            country = str(arr[1]).replace("'","").replace("´","")
            insertsql = ('INSERT INTO countries VALUES'+("('"+str(arr[0])+"' , '"+country+"')"))
            cursorObj.execute(insertsql)
            con.commit()
            barposition+=1
            bar.update(i)

#### Define the conection to the db and create the cursor

In [ ]:
con = pymysql.connect(host='77.240.114.226',
                     user='wpp_challenge',
                     password='Qmi2b3?2',
                     db='wpp_challenge')
cursorObj = con.cursor()

#### Create the tables, if it was created before, don't do nothing

In [ ]:
#Create the table for the dftags.
try: 
    sql_tabletag(con)
    print ('Table TAG created')    
except: 
    print ('Table TAG already exist')

#Create the table for the dftags.
try: 
    sql_tablefull(con)
    print ('Table fulldata  created')

except: 
    print ('Table FULL DATA already exist')

#Create the table for the resume.
try: 
    sql_tablesummary(con)
    print ('Table RESUME  created')

except: 
    print ('Table RESUME already exist')
    
try: 
    sql_tabledates(con)
    print ('Table DATES  created')

except: 
    print ('Table DATES already exist')

try: 
    sql_tablecountries(con)
    print ('Table COUNTRIES  created')

except: 
    print ('Table COUNTRIES already exist')

#### Now, we use the data from the different df's to upload our tables

In [ ]:
insertDFtags(dftags, con)
insertDFfull(dfF, con)
insertDFdates(dfdates, con)
insertDFcountries(dfcountries, con)
summary(con)